In [2]:
# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip

In [3]:
# !unzip data.zip

In [4]:
# !ls

In [5]:
DIR_TRAIN_BEE='data/train/bee/'
DIR_TRAIN_WASP='data/train/wasp/'
DIR_TRAIN='data/train/'
DIR_TEST_BEE='data/test/bee/'
DIR_TEST_WASP='data/test/wasp/'
DIR_TEST='data/test/'
input_shape=(150,150)

In [6]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-11-22 19:40:31.850177: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 19:40:31.850218: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 19:40:31.850230: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 19:40:31.854410: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
def create_model():
   model = models.Sequential()
   model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
   model.add(layers.MaxPooling2D((2, 2)))
   model.add(layers.Flatten())
   model.add(layers.Dense(64, activation='relu'))
   model.add(layers.Dense(1, activation='sigmoid'))
   
   sgd = tf.keras.optimizers.SGD(learning_rate=0.002, momentum=0.8)
   
   model.compile(optimizer=sgd,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
   
   return model


In [61]:
model = create_model()

In [62]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 74, 74, 32)        0         
 ng2D)                                                           
                                                                 
 flatten_10 (Flatten)        (None, 175232)            0         
                                                                 
 dense_20 (Dense)            (None, 64)                11214912  
                                                                 
 dense_21 (Dense)            (None, 1)                 65        
                                                                 
Total params: 11215873 (42.79 MB)
Trainable params: 11215873 (42.79 MB)
Non-trainable params: 0 (0.00 Byte)
___________

In [63]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
   DIR_TRAIN,
   target_size=(150, 150),
   batch_size=20,
   class_mode='binary',
   shuffle=True)

test_generator = test_datagen.flow_from_directory(
   DIR_TEST,
   target_size=(150, 150),
   batch_size=20,
   class_mode='binary',
   shuffle=True)


Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [64]:
history=model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
184/184 [==============================] - 3s 15ms/step - loss: 0.6846 - accuracy: 0.5409 - val_loss: 0.6670 - val_accuracy: 0.5861
Epoch 2/10
184/184 [==============================] - 3s 16ms/step - loss: 0.6554 - accuracy: 0.5891 - val_loss: 0.6797 - val_accuracy: 0.5806
Epoch 3/10
184/184 [==============================] - 3s 16ms/step - loss: 0.6279 - accuracy: 0.6413 - val_loss: 0.5999 - val_accuracy: 0.6492
Epoch 4/10
184/184 [==============================] - 3s 15ms/step - loss: 0.5802 - accuracy: 0.6905 - val_loss: 0.5631 - val_accuracy: 0.7113
Epoch 5/10
184/184 [==============================] - 3s 15ms/step - loss: 0.5405 - accuracy: 0.7392 - val_loss: 0.5474 - val_accuracy: 0.7429
Epoch 6/10
184/184 [==============================] - 3s 15ms/step - loss: 0.5152 - accuracy: 0.7580 - val_loss: 0.5514 - val_accuracy: 0.7026
Epoch 7/10
184/184 [==============================] - 3s 16ms/step - loss: 0.4923 - accuracy: 0.7778 - val_loss: 0.5234 - val_accuracy: 0.7538

In [65]:
# Get the list of accuracy values for each epoch
accuracy_values = history.history['accuracy']

# Calculate the median of the accuracy values
median_accuracy = np.median(accuracy_values)

print("Median training accuracy: ", median_accuracy)

Median training accuracy:  0.7485722005367279


In [66]:
loss_values = history.history['loss']

# Calculate the standard deviation of the loss values
std_dev_loss = np.std(loss_values)

print("Standard deviation of training loss: ", std_dev_loss)

Standard deviation of training loss:  0.08186884051831032


In [67]:
train_datagen_ag = ImageDataGenerator(rescale=1./255,
                rotation_range=50,
                width_shift_range=0.1,
                height_shift_range=0.1,
                zoom_range=0.1,
                horizontal_flip=True,
                fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator_ag = train_datagen_ag.flow_from_directory(
   DIR_TRAIN,
   target_size=(150, 150),
   batch_size=20,
   class_mode='binary',
   shuffle=True)

test_generator = test_datagen.flow_from_directory(
   DIR_TEST,
   target_size=(150, 150),
   batch_size=20,
   class_mode='binary',
   shuffle=True)


Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [68]:
history=model.fit(
    train_generator_ag,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
184/184 [==============================] - 10s 51ms/step - loss: 0.5013 - accuracy: 0.7691 - val_loss: 0.5010 - val_accuracy: 0.7593
Epoch 2/10
184/184 [==============================] - 9s 49ms/step - loss: 0.4896 - accuracy: 0.7764 - val_loss: 0.4761 - val_accuracy: 0.7691
Epoch 3/10
184/184 [==============================] - 9s 50ms/step - loss: 0.4881 - accuracy: 0.7732 - val_loss: 0.4757 - val_accuracy: 0.7810
Epoch 4/10
184/184 [==============================] - 9s 50ms/step - loss: 0.4893 - accuracy: 0.7721 - val_loss: 0.5312 - val_accuracy: 0.7397
Epoch 5/10
184/184 [==============================] - 9s 50ms/step - loss: 0.4768 - accuracy: 0.7819 - val_loss: 0.5088 - val_accuracy: 0.7614
Epoch 6/10
184/184 [==============================] - 9s 50ms/step - loss: 0.4725 - accuracy: 0.7819 - val_loss: 0.5248 - val_accuracy: 0.7527
Epoch 7/10
184/184 [==============================] - 9s 50ms/step - loss: 0.4639 - accuracy: 0.7871 - val_loss: 0.4867 - val_accuracy: 0.774

In [69]:
val_loss_values = history.history['val_loss']


mean_val_loss = np.mean(val_loss_values)

print("Mean validation loss: ", mean_val_loss)

Mean validation loss:  0.4918523639440536


In [70]:
test_accuracy_values = history.history['val_accuracy'][5:]

np.mean(test_accuracy_values)

0.772113299369812

In [53]:
history.history

{'loss': [0.6910605430603027,
  0.6753612160682678,
  0.6544091105461121,
  0.6229560971260071,
  0.5885480642318726,
  0.5679607391357422,
  0.5465046763420105,
  0.5441578030586243,
  0.5232158899307251,
  0.5248780846595764],
 'accuracy': [0.5392983555793762,
  0.5700299143791199,
  0.6007614731788635,
  0.6521620750427246,
  0.6877889633178711,
  0.7111775875091553,
  0.7359260320663452,
  0.7478923201560974,
  0.7614903450012207,
  0.7552352547645569],
 'val_loss': [0.707446277141571,
  0.6512184143066406,
  0.6149158477783203,
  0.5774254202842712,
  0.569033145904541,
  0.5411583781242371,
  0.5235651135444641,
  0.5278999209403992,
  0.5068790912628174,
  0.5148460268974304],
 'val_accuracy': [0.5370370149612427,
  0.56971675157547,
  0.6623093485832214,
  0.6960784196853638,
  0.6895424723625183,
  0.7549019455909729,
  0.7603485584259033,
  0.7331154942512512,
  0.7614378929138184,
  0.7494553327560425]}